In [1]:
import time
import pandas as pd
import os
from selenium import webdriver

In [2]:
# 기사정보를 저장하는 함수
def article_save(query1, query2, query3, ds, de):
    # 기사제목, 링크주소, 좋아요, 화나요를 저장할 딕셔너리
    article_dict = { 'title':[], 'link':[], 'like':[], 'angry':[] }
    
    # article_search()에서 입력한 조건으로 기사를 검색한다.
    # 한 페이지당 큰 제목 기사 10개씩
    for start in range(1,92,10) :
        site = f'https://search.naver.com/search.naver?where=news&query={query1}%26{query2}%26{query3}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds}&de={de}&docid=&nso=so%3Ar%2Cp%3Afrom20190101to20200206%2Ca%3Aall&mynews=0&&start={start}&related=0'
        driver.get(site)
        time.sleep(1)
        for i in range(0, 10, 1) :
            # 기사 내용 중에 (큰 제목만 탐색)
            # 네이버뉴스 링크가 있으면 
            # 딕셔너리에 기사제목과 링크주소 추가 후
            # 기사를 열어본다.
            content_list = driver.find_elements_by_css_selector('ul > li > dl')
            content = content_list[i]
            if ('네이버뉴스' in content.text) == True :
                # 기사 제목과 링크 딕셔너리에 추가
                title = content.find_element_by_css_selector('dt')
                try:
                    link = content.find_element_by_css_selector('dd > a')
                # 마지막 페이지 오류 발생시 예외처리
                except:
                    print('--마지막 페이지--')
                    pass
                else:
                    article_dict['title'].append(title.text.strip(''))
                    article_dict['link'].append(link.get_attribute('href'))
                    link.click()

                    # 네이버뉴스가 열린 탭으로 이동
                    news_tab = driver.window_handles[-1]
                    driver.switch_to.window(window_name=news_tab)

                    # 좋아요 수와 화나요 수를 가져온다.(1000이상이면 콤마 제거)
                    like = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
                    angry = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')
                    article_dict['like'].append(like.text.replace('[\,]', ''))
                    article_dict['angry'].append(angry.text.replace('[\,]', ''))

                    # 기사 탭을 닫고 검색결과 화면으로 돌아간다.
                    driver.close()
                    first_tab = driver.window_handles[0]
                    driver.switch_to.window(window_name=first_tab)
            
            else:
                pass
                
        # 내부for문 종료
    
        # 데이터프레임으로 추가
        df = pd.DataFrame.from_dict(article_dict)
        df.to_csv('naver_article.csv', encoding='utf-8-sig', mode='w', index=False)
        print('페이지 탐색 완료')
        
    # 외부 for문 종료
    return print('수집완료')

In [3]:
driver = webdriver.Chrome()

# 네이버뉴스를 검색하는 함수
def article_search():
    # 검색어, 기간을 입력한다.
    # 날짜 형식 : yyyy.mm.dd
    query1 = input('키워드1 : ')
    query2 = input('키워드2 : ')
    query3 = input('키워드3 : ')
    ds = input('시작일 : ')
    de = input('종료일 : ')
    
    # 기사 저장하는 함수 호출
    article_save(query1, query2, query3, ds, de)

In [4]:
article_search()

키워드1 : 강남구
키워드2 : 재개발
키워드3 : 재건축
시작일 : 2018.01.01
종료일 : 2019.01.01
마지막 페이지입니다
페이지 탐색 완료
수집완료
